In [26]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [27]:
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [28]:
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='./data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [29]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [30]:
net = ConvNet(num_classes)

In [31]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [32]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images
        labels = labels
        
        # Forward pass
        outputs = net(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.2104
Epoch [1/5], Step [200/600], Loss: 0.1556
Epoch [1/5], Step [300/600], Loss: 0.1090
Epoch [1/5], Step [400/600], Loss: 0.0619
Epoch [1/5], Step [500/600], Loss: 0.0702
Epoch [1/5], Step [600/600], Loss: 0.0413
Epoch [2/5], Step [100/600], Loss: 0.0602
Epoch [2/5], Step [200/600], Loss: 0.1152
Epoch [2/5], Step [300/600], Loss: 0.0191
Epoch [2/5], Step [400/600], Loss: 0.0092
Epoch [2/5], Step [500/600], Loss: 0.0523
Epoch [2/5], Step [600/600], Loss: 0.0400
Epoch [3/5], Step [100/600], Loss: 0.0096
Epoch [3/5], Step [200/600], Loss: 0.0241
Epoch [3/5], Step [300/600], Loss: 0.0226
Epoch [3/5], Step [400/600], Loss: 0.0056
Epoch [3/5], Step [500/600], Loss: 0.0196
Epoch [3/5], Step [600/600], Loss: 0.0464
Epoch [4/5], Step [100/600], Loss: 0.0061
Epoch [4/5], Step [200/600], Loss: 0.0302
Epoch [4/5], Step [300/600], Loss: 0.0301
Epoch [4/5], Step [400/600], Loss: 0.0516
Epoch [4/5], Step [500/600], Loss: 0.0283
Epoch [4/5], Step [600/600], Loss:

In [34]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images
        labels = labels
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 98.85 %
